Original Code

In [3]:
#Import the needed modules
import pya
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Tester_Pattern.gds', mesh:str = 'Test_Mesh_Small.csv', target_layer:int = 0, LL:list=(0,0), UR:list=(382, 490),output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Import the .gds
    gds = pya.Layout()
    gds.read(gds_file)
    print('File read...')

    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Hard coding layer levels (this can be improved in the future...)
    bias_lyr_index = 42
    final_lyr_index = 43

    #Create bias layer for eventual use
    bias_lyr = gds.layer(bias_lyr_index,0,'Initial Biasing Layer')

    #Create final bias layer
    final_lyr = gds.layer(final_lyr_index,0,'Final Biased Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print('Mesh read...')

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells)
    top_cell = gds.top_cell()
    print('Top cell chosen, entering loop...')

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in tqdm(rows):
        for j in tqdm(columns):
                #Define the edges of the mesh at each location
                l_edge = LL[0]+mesh_x*(j-1)
                b_edge = UR[1]-mesh_y*i
                r_edge = LL[0]+mesh_x*j
                t_edge = UR[1]-mesh_y*(i-1)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]

                #Takes polygons from the copy layer in a given search region to keep in the "touching" pya.Region 
                search_region = pya.DBox(l_edge,b_edge,r_edge,t_edge)
                touching = pya.Region(top_cell.begin_shapes_rec_touching(target_layer, search_region))

                #Biases "touching" and copies it into the top cell in the bias layer
                touch_up = touching.sized(int(biasing))
                top_cell.shapes(bias_lyr).insert(touch_up)
                
    
    print("Finished Mesh Loop")

    #Merges any overlapping polygons and places the final biased data onto the "final" layer
    bias_merge = pya.Region(top_cell.begin_shapes_rec_touching(bias_lyr,pya.DBox(LL[0],LL[1],UR[0],UR[1])))
    bias_merge.merge()
    top_cell.shapes(final_lyr).insert(bias_merge)

    #Removes unneeded bias layer from the layout
    gds.clear_layer(bias_lyr)
    

    #Writes the new file
    return gds.write(output_file)




rpdb()



File read...
Mesh read...
There are 21 columns in the mesh
There are 21 rows in the mesh
Mesh X is 19.1nm
Mesh Y is 24.5nm
Top cell chosen, entering loop...


100%|██████████| 20/20 [00:02<00:00,  9.31it/s]

Finished Mesh Loop


New Test (trying with clips)

In [15]:
#Import the needed modules
import klayout.db as db
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Poly_tester.gds', mesh:str = 'Test_Mesh_Small.csv', target_layer:int = 0,output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased

    #Import the .gds
    gds = db.Layout()
    gds.read(gds_file)
    print(f'File read: {gds_file}')

    #Sets the database units as 1nm
    gds.dbu = 0.001
    
    #Create bias layer for eventual use
    bias_lyr = gds.layer('Initial Biasing Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print(f'Mesh read: {mesh}')

    #Determine maximum absolute bias applied, in order to size up the cell regions
    mesh_max = pdb.max().max()
    mesh_min = pdb.min().min()
    if abs(mesh_max) >= abs(mesh_min):
         bias_max = mesh_max/1000 #to get in units of um
    else:
         bias_max = abs(mesh_min) #to get in units of um

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')
    
    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells) and stores index of all called cells
    top_cell = gds.top_cell()
    top_cell_index = top_cell.cell_index()
    called_cells = top_cell.called_cells()
    
    #Determines layout size
    layout_bounding = top_cell.dbbox()
    left = layout_bounding.left
    bottom = layout_bounding.bottom
    right = layout_bounding.right
    top = layout_bounding.top

    print(f'{left},{bottom},{right},{top}')
    
    #Determine layer sizing
    x_size = right - left
    y_size = top - bottom

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Create new top cell
    New_Top = gds.create_cell("NewTop")
    New_Top_Index = New_Top.cell_index()

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in tqdm(rows):
        for j in columns:
                #Define the edges of the mesh at each location
                l_edge = left+(mesh_x*(j-1)-bias_max)
                b_edge = top-(mesh_y*i+bias_max)
                r_edge = left+(mesh_x*j+bias_max)
                t_edge = top-(mesh_y*(i-1)-bias_max)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]
                #print(f'This is the bias value: {biasing}')

                #Clips mesh area from the polygon, preserving hierarchy, and names it
                clip = gds.clip(top_cell,db.DBox(l_edge,b_edge,r_edge,t_edge))
                clip.name= f'Mesh_Cell_{i}_{j}'
                
                #Copies the cell tree from the clip into a new bias cell
                bias_cell = gds.create_cell(f'bias_cell_{i}_{j}')
                bias_tree = bias_cell.copy_tree(clip)

                #Goes through the hierarchy of each clip and biases by the appropriate value, and inserts into the bias layer
                for k in bias_cell.called_cells():
                    copy_cell = gds.cell(k)
                    search_region = db.Region(copy_cell.bbox(target_layer))
                    shapes_region = db.Region(copy_cell.shapes(target_layer))
                    touching = search_region & shapes_region
                    bias_apply = touching.sized(int(biasing))
                    copy_cell.shapes(bias_lyr).insert(bias_apply)


                #Insert the new bias tree into the New Top Cell, and prune the clip and bias cell                                                                
                New_Top.copy_tree(bias_cell)
                bias_cell.prune_cell()
                clip.prune_cell()                    
    
    print("Finished Mesh Loop")

    #Remove the old top cell
    top_cell.prune_cell()

    #Writes the new file
    return gds.write(output_file)

rpdb()

File read: Poly_tester.gds
Mesh read: Test_Mesh_Small.csv
There are 21 columns in the mesh
There are 21 rows in the mesh
-0.085,-10.433,388.64300000000003,323.351
Mesh X is 19.4364nm
Mesh Y is 16.6892nm


100%|██████████| 20/20 [00:02<00:00,  8.18it/s]

Finished Mesh Loop


Testing with generators / list comprehensions

In [48]:
#Import the needed modules
import klayout.db as db
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Poly_tester.gds', mesh:str = 'Test_Mesh_Small.csv', target_layer:int = 0,output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased

    #Import the .gds
    gds = db.Layout()
    gds.read(gds_file)
    print(f'File read: {gds_file}')

    #Sets the database units as 1nm
    gds.dbu = 0.001
  
    #Create bias layer for eventual use
    bias_lyr = gds.layer('Initial Biasing Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print(f'Mesh read: {mesh}')

    #Determine maximum absolute bias applied, in order to size up the cell regions
    mesh_max = pdb.max().max()
    mesh_min = pdb.min().min()
    if abs(mesh_max) >= abs(mesh_min):
         bias_max = mesh_max/1000 #to get in units of um
    else:
         bias_max = abs(mesh_min/1000) #to get in units of um

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')
    
    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells) and stores index of all called cells
    top_cell = gds.top_cell()
    top_cell_index = top_cell.cell_index()
    called_cells = top_cell.called_cells()
    
    #Determines layout size
    layout_bounding = top_cell.dbbox()
    left = layout_bounding.left
    bottom = layout_bounding.bottom
    right = layout_bounding.right
    top = layout_bounding.top

    print(f'{left},{bottom},{right},{top}')
    
    #Determine layer sizing
    x_size = right - left
    y_size = top - bottom

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Define the edges of the mesh at each location
    l_edge = [left+(mesh_x*(j-1)-bias_max) for j in columns]
    b_edge = [top-(mesh_y*i+bias_max) for i in rows]
    r_edge = [left+(mesh_x*j+bias_max) for j in columns]
    t_edge = [top-(mesh_y*(i-1)-bias_max) for i in rows]

    print(l_edge, b_edge, r_edge, t_edge)

    #Define the biasing at each location
    new_j = [str(j) for j in columns]
    new_i = [i for i in rows]
    biasing = [[pdb[j][i] for i in new_i] for j in new_j]
    print(biasing)

    #Clips mesh area from the polygon, preserving hierarchy, and names it
    clip = gds.clip(top_cell,db.DBox(l_edge,b_edge,r_edge,t_edge))
    clip.name = f'Mesh_Cell_{i}_{j}'

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    # for i in tqdm(rows):
    #     for j in columns:

    #             #Clips mesh area from the polygon, preserving hierarchy, and names it
    #             clip = gds.clip(top_cell,db.DBox(l_edge,b_edge,r_edge,t_edge))
    #             clip.name= f'Mesh_Cell_{i}_{j}'
                

    #             #Loop to identify each shape and size it up by the bias value amount
    #             #print(f'Clip called cells are: {clip.called_cells()}')

    #             #Copies the cell tree from the clip into a new bias cell
    #             bias_cell = gds.create_cell(f'bias_cell_{i}_{j}')
    #             bias_tree = bias_cell.copy_tree(clip)
    #             #print(f'Biasing is {biasing}')

    #             #Goes through the hierarchy of each clip and biases by the appropriate value, and inserts into the bias layer
    #             for k in bias_cell.called_cells():
    #                 copy_cell = gds.cell(k)
    #                 search_region = db.Region(copy_cell.bbox(target_layer))
    #                 shapes_region = db.Region(copy_cell.shapes(target_layer))
    #                 touching = search_region & shapes_region
    #                 bias_apply = touching.sized(int(biasing))
    #                 copy_cell.shapes(bias_lyr).insert(bias_apply)
                                                     
    #             #inserter = db.CellInstArray(bias_cell,0,0,0,0,0)
    #             #top_cell.insert(inserter)   
    #             clip.prune_cell()                    
    
    # print("Finished Mesh Loop")
    
    # #Writes the new file
    # return gds.write(output_file)

rpdb()

File read: Poly_tester.gds
Mesh read: Test_Mesh_Small.csv
There are 21 columns in the mesh
There are 21 rows in the mesh
-0.085,-10.433,388.64300000000003,323.351
Mesh X is 19.4364nm
Mesh Y is 16.6892nm
[-0.10500000000000001, 19.3314, 38.767799999999994, 58.20419999999999, 77.6406, 97.077, 116.5134, 135.94979999999998, 155.38619999999997, 174.82259999999997, 194.25899999999996, 213.69539999999998, 233.13179999999997, 252.56819999999996, 272.00460000000004, 291.44100000000003, 310.8774, 330.3138, 349.7502, 369.1866] [306.6418, 289.9526, 273.2634, 256.5742, 239.885, 223.19580000000002, 206.5066, 189.8174, 173.1282, 156.439, 139.7498, 123.0606, 106.3714, 89.6822, 72.993, 56.303800000000024, 39.614599999999996, 22.925400000000025, 6.2362000000000535, -10.452999999999975] [19.371399999999998, 38.8078, 58.2442, 77.6806, 97.11699999999999, 116.5534, 135.9898, 155.4262, 174.8626, 194.29899999999998, 213.7354, 233.1718, 252.60819999999998, 272.0446, 291.481, 310.9174, 330.3538, 349.790199999999